### References:
* [docling_rag repository by shashanka300](https://github.com/shashanka300/docling_rag?utm_source=pocket_shared)
* [multimodal RAG system with Docling and Granite](https://www.ibm.com/think/tutorials/build-multimodal-rag-langchain-with-docling-granite?utm_source=pocket_saves)
* [Llama_index example with RAG and Reraking workflow](https://docs.llamaindex.ai/en/stable/examples/workflow/rag/)

In [1]:
import converter

In [2]:
processor = converter.DocumentProcessor(summarizer_model='gemma3:4b')

In [3]:
conversions = processor.convert('/Users/rauldemaio/Projects Local/agent_rag/data/fattura.pdf')

100%|██████████| 1/1 [00:11<00:00, 11.01s/it]


In [4]:
nodes, mapping = processor.chunk_documents(conversions)

Usage of TableItem.export_to_markdown() without `doc` argument is deprecated.
Usage of TableItem.export_to_markdown() without `doc` argument is deprecated.
Usage of TableItem.export_to_markdown() without `doc` argument is deprecated.
Usage of TableItem.export_to_markdown() without `doc` argument is deprecated.


In [5]:
nodes = processor.add_embeddings(nodes, method='summary')

100%|██████████| 9/9 [00:55<00:00,  6.22s/it]


In [6]:
from llama_index.llms.ollama import Ollama
from StructuredResponse import InvoiceResponse
sllm = Ollama(
    model='phi4-mini:latest',
    request_timeout=600.0, 
    temeperature=0.01,
    context_window=124000
).as_structured_llm(InvoiceResponse)

/opt/anaconda3/envs/llm_agent/lib/python3.13/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [7]:
from llama_index.core.vector_stores import SimpleVectorStore
from llama_index.core import VectorStoreIndex
#vectorstore = SimpleVectorStore()
#vectorstore.add(nodes)

index = VectorStoreIndex(nodes, embed_model=processor.embed_model)
#query_engine = index.as_query_engine(similarity_top_k=5, response_mode="refine",llm = sllm)
#query_engine.query("A chi è indirizzata la fattura?")

In [8]:
from RAGWorkflow import RAGWorkflow
w = RAGWorkflow()

In [9]:
result = await w.run(
    query="What is the total cost, included VAT, for the purchased items?",
    index=index,
    processor=processor,
    sllm=sllm,
)

async for chunk in result.async_response_gen():
    print(chunk, end="", flush=True)

Query the database with: What is the total cost, included VAT, for the purchased items?
Retrieved 5 nodes.
What is the total cost, included VAT, for the purchased items?
Reranked nodes to 1


WorkflowTimeoutError: Operation timed out after 10.0 seconds

In [12]:
result = await w.run(
    query="Which items have been purchased? Answer in italian.",
    index=index,
    processor=processor,
    sllm=sllm,
)

async for chunk in result.async_response_gen():
    print(chunk, end="", flush=True)

Query the database with: Which items have been purchased? Answer in italian.
Retrieved 5 nodes.
Which items have been purchased? Answer in italian.
Reranked nodes to 2


WorkflowRuntimeError: Error in step 'synthesize': 1 validation error for InvoiceResponse
  Invalid JSON: expected value at line 1 column 1 [type=json_invalid, input_value='<async_generator object ...ls>.gen at 0x3c07edcb0>', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/json_invalid

In [12]:
result = await w.run(
    query="Who is the owner of the invoice? Answer in italian.",
    index=index,
    processor=processor,
)

async for chunk in result.async_response_gen():
    print(chunk, end="", flush=True)

Query the database with: Who is the owner of the invoice? Answer in italian.
Retrieved 5 nodes.
Who is the owner of the invoice? Answer in italian.
Reranked nodes to 2
L'intestatario della fattura è indicato come “CSE003”.

In [13]:
result = await w.run(
    query="Which is the shipping or delivery address? What is the delivery and invoice dates? Are they different? Answer in italian.",
    index=index,
    processor=processor,
)

async for chunk in result.async_response_gen():
    print(chunk, end="", flush=True)

Query the database with: Which is the shipping or delivery address? What is the delivery and invoice dates? Are they different? Answer in italian.
Retrieved 5 nodes.
Which is the shipping or delivery address? What is the delivery and invoice dates? Are they different? Answer in italian.
Reranked nodes to 2
L'indirizzo di consegna è Via vestricio spurinna 57 roma, 00175 ITA. Non sono presenti date di fattura o di consegna per determinare se sono diverse.

In [15]:
result = await w.run(
    query="What is the delivery and invoice dates? Are they different? Answer in italian.",
    index=index,
    processor=processor,
)

async for chunk in result.async_response_gen():
    print(chunk, end="", flush=True)

Query the database with: What is the delivery and invoice dates? Are they different? Answer in italian.
Retrieved 5 nodes.
What is the delivery and invoice dates? Are they different? Answer in italian.
Reranked nodes to 2
La data di spedizione è il 23/09/2024. La data della fattura è il 23/09/2024. Sono uguali.

---

Alternativa:
1. Leggi il documento con docling
2. Estrai il markdown
    * Usa un Markdown Node Parser per ottenere i nodi testuali
3. Estrai le tabelle
    * Costruisci l'index object

Altrimenti usare [hierarchical Chunker di docling](https://docling-project.github.io/docling/examples/rag_llamaindex/) su cui però non c'è particolare controllo. 


Una volta ottenuto l'index e il retriever, si può optare per un [https://docs.llamaindex.ai/en/stable/use_cases/agents/](agent con llama_index)

In [1]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

EMBED_MODEL = HuggingFaceEmbedding('sentence-transformers/all-MiniLM-L6-v2')
SOURCE = '/Users/rauldemaio/Projects Local/agent_rag/data/fattura.pdf'

In [ ]:
from docling.document_converter import DocumentConverter
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.core.node_parser import MarkdownNodeParser

reader = DocumentConverter()
doc = reader.convert(SOURCE).document

In [43]:
text = doc.export_to_markdown()
table_list = [(table.export_to_markdown(),table.export_to_dataframe()) for table in doc.tables]

Usage of TableItem.export_to_markdown() without `doc` argument is deprecated.
Usage of TableItem.export_to_markdown() without `doc` argument is deprecated.
Usage of TableItem.export_to_markdown() without `doc` argument is deprecated.
Usage of TableItem.export_to_markdown() without `doc` argument is deprecated.


In [46]:
text

'<!-- image -->\n\nCliente: CSE003 Raul De Maio     Data: 23/9/2024     Numero fattura: E00324337228\n\nNumero ordine web:  IT4453739\n\n## Spett.le:\n\nRaul De Maio\n\n## Indirizzo di fatturazione :\n\nVia vestricio spurinna  57 roma, 00175 ITA\n\n## Indirizzo di consegna:\n\nVia vestricio spurinna  57 roma, 00175 ITA\n\nPeso netto:\n\nPeso lordo:\n\nVolume:\n\nColli:\n\n0,50\n\n0,50\n\n1000000000000000,00\n\n1,00\n\n## Tipo documento: Documento commerciale\n\n| Numero:                          | Data:                           | Conto fattura:              | Partita IVA :        |\n|----------------------------------|---------------------------------|-----------------------------|----------------------|\n| E00324337228                     | 23/9/2024                       | CSE003                      |                      |\n| Condizioni di Pagamento:         | TP01  ANT                       |                             |                      |\n| Nome della banca:               

In [52]:
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.core import Document

documents = [Document(text=text)]

parser = MarkdownNodeParser()

In [53]:
parser.get_nodes_from_documents(documents)

[TextNode(id_='ee20eb25-e420-4f31-aac0-f8b1b7204d39', embedding=None, metadata={'header_path': '/'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='5b7d7129-5b4d-42c1-acbe-38d76b69f8e9', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='89fec49ef2418489cdb6de539d8b3032a1a8cc3088b31a019771330c7e5bb156'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='fb8dd2ec-eaf5-4370-bd2f-6a35c27b3329', node_type=<ObjectType.TEXT: '1'>, metadata={'header_path': '/'}, hash='d7aef55f01f31e74d8f1b2091b215b1dc29bc06dd21d17a68c02fa04171cbf76')}, metadata_template='{key}: {value}', metadata_separator='\n', text='<!-- image -->\n\nCliente: CSE003 Raul De Maio     Data: 23/9/2024     Numero fattura: E00324337228\n\nNumero ordine web:  IT4453739', mimetype='text/plain', start_char_idx=0, end_char_idx=128, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'),
 TextNode(id_='fb8dd2ec-eaf5-4370

----

First Version Without Source code in .py file

In [ ]:
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import (
    AcceleratorDevice,
    AcceleratorOptions,
    PdfPipelineOptions,
    TableFormerMode
)
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling_core.transforms.chunker.hybrid_chunker import HybridChunker
from sentence_transformers import SentenceTransformer

from typing import List, Dict, Any
from tqdm.notebook import tqdm

In [ ]:
pipeline_options = PdfPipelineOptions()
pipeline_options.do_ocr = True
pipeline_options.do_table_structure = True
pipeline_options.table_structure_options.do_cell_matching = True
pipeline_options.ocr_options.lang = ["en","it"]
pipeline_options.table_structure_options.mode = TableFormerMode.ACCURATE
pipeline_options.accelerator_options = AcceleratorOptions(
    num_threads=8, device=AcceleratorDevice.MPS
)

converter = DocumentConverter(
            format_options={
                InputFormat.PDF: PdfFormatOption(
                    pipeline_options=pipeline_options
                )
            }
        )

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding('sentence-transformers/all-MiniLM-L6-v2')

# embed_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
sources = ['/Users/rauldemaio/Projects Local/agent_rag/data/fattura.pdf']
conversions = {}

for source in tqdm(sources):
    # Convert the document to a Document object
    conversions[source] = converter.convert(source=source).document


In [ ]:
from docling_core.transforms.chunker.hybrid_chunker import HybridChunker
from docling_core.types.doc.document import TableItem

doc_id = 0



In [ ]:
def extract_chunk_metadata(chunk) -> Dict[str, Any]:
    """Extract essential metadata from a chunk"""
    metadata = {
        "text": chunk.text,
        "headings": [],
        "page_info": None,
        "content_type": None
    }
        
    if hasattr(chunk, 'meta'):
        # Extract headings
        if hasattr(chunk.meta, 'headings') and chunk.meta.headings:
            metadata["headings"] = chunk.meta.headings
            
        # Extract page information and content type
        if hasattr(chunk.meta, 'doc_items'):
            for item in chunk.meta.doc_items:
                if hasattr(item, 'label'):
                    metadata["content_type"] = str(item.label)
                    
                if hasattr(item, 'prov') and item.prov:
                    for prov in item.prov:
                        if hasattr(prov, 'page_no'):
                            metadata["page_info"] = prov.page_no
        
    return metadata

In [ ]:
from llama_index.core.schema import TextNode, NodeRelationship, RelatedNodeInfo

nodes = []
chunker = HybridChunker(tokenizer="jinaai/jina-embeddings-v3")

for source, docling_document in conversions.items():
    for chunk in chunker.chunk(docling_document):
        
        items = chunk.meta.doc_items

        if len(items) == 1 and isinstance(items[0], TableItem):

            continue # we will process tables later

        refs = " ".join(map(lambda item: item.get_ref().cref, items))

        print(refs)

        text = chunk.text
        chunk_metadata = extract_chunk_metadata(chunk)

        node = TextNode(
            id_=str(doc_id+1),
            text=text,
            #embedding = embed_model.encode(text),
            metadata ={
                'source':source,
                'ref':refs,
                "page_info": chunk_metadata["page_info"],
                "content_type": chunk_metadata["content_type"],
                "headings": chunk_metadata["headings"]
                
            }
        )

        nodes.append(node)

        doc_id += 1
        # Add relationships
        



In [ ]:
from docling_core.types.doc.labels import DocItemLabel

doc_id = len(nodes)

tables = []

for source, docling_document in conversions.items():

    for table in docling_document.tables:

        if table.label in [DocItemLabel.TABLE]:

            ref = table.get_ref().cref

            print(ref)

            text = table.export_to_markdown()

            #chunk_metadata = extract_chunk_metadata(chunk)

            node = TextNode(
            id_=str(doc_id+1),
            text=text,
            #embedding = embed_model.encode(text),
            metadata ={
                'source':source,
                'ref':refs,
                #"page_info": chunk_metadata["page_info"],
                "content_type": 'TABLE',
                #"headings": chunk_metadata["headings"]
                #'data': table.export_to_dataframe()
            }
        )

        tables.append(node)

        doc_id += 1

            

In [ ]:
import itertools

for document in itertools.chain(nodes, tables):

    print(f"Document ID: {document.id_}")

    print(f"Source: {document.metadata['source']}")

    print(f"Content Type: {document.metadata['content_type']}")

    print(f"Content:\n{document.text[:50]}...")

    print("=" * 80) # Separator for clarity



In [ ]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama

Settings.llm = Ollama(model="gemma3:4b", request_timeout=60.0)

In [ ]:
from llama_index.core.vector_stores import SimpleVectorStore

vectorstore = SimpleVectorStore()

In [ ]:
for node in nodes:

    node.embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode='all')
        )

In [ ]:
for table in tables:
    table.embedding = embed_model.get_text_embedding(
        table.get_content()
        )

In [ ]:
vectorstore.add(list(itertools.chain(nodes, tables)))

In [ ]:
index = VectorStoreIndex(list(itertools.chain(nodes, tables)), embed_model=embed_model)

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=3, response_mode="tree_summarize")

In [ ]:
response = query_engine.query("Quale è la data della spedizione?")
display(response)
for node in response.source_nodes:
    print(f"Node ID: {node.id_}")
    print(f"Source: {node.metadata['source']}")
    print(f"Content Type: {node.metadata['content_type']}")
    print(f"Content:\n{node.text}.")
    print(f"Score: {node.score}")
    print("=" * 80)  # Separator for clarity

In [ ]:
# aggiungere un summarizer alle tabelle
# aggiungere reranking
# adottare un agent tool o un workflow
